# Procesado Espectros Raman Sonda 532nm

In [1]:
import numpy as np
import pandas as pd
#%matplotlib inline
import re
import os
import math
import numpy as np
import pandas as pd
import joblib
import warnings
import time
import matplotlib.pyplot as plt

In [13]:
# Ruta que contiene todos los archivos
folder_path = "D:/OneDrive - UNICAN/LIBS/raman/ConchasArq"
# Directorios de medida
directories = [directory for directory in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, directory)) and directory != "Fotos"]


No exported files found in 09042024-160856_conchas_532nm_LIT1_n10_1000ms_0.05mm_50p_atras
Processing 10042024-103809_conchas_532nm_LIT2_n10_1000ms_0.05mm_50p_atras


ParserError: Error tokenizing data. C error: Expected 1044 fields in line 3, saw 1046


In [22]:
# Ruta que contiene todos los archivos
folder_path = "D:/OneDrive - UNICAN/LIBS/raman/ConchasArq"
# Directorios de medida
directories = [directory for directory in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, directory)) and directory != "Fotos"]

for directory in directories :
    archivos = os.listdir(os.path.join(folder_path, directory))
    print(f"Directorio {directory} contiene {archivos}")

directories

directories_test = ['09042024-160856_conchas_532nm_LIT1_n10_1000ms_0.05mm_50p_atras','10042024-103809_conchas_532nm_LIT2_n10_1000ms_0.05mm_50p_atras',
 '10042024-152046_conchas_532nm_LIT3_n10_1000ms_0.05mm_50_atras',
 '11042024-162047_conchas_532nm_LIT4_n10_1000ms_0.05mm_50p_caraabajo',
 '17042024-150606_conchas_532nm_LIT5_n10_1000ms_0.05mm_50p_atras',
 '17042024-184049_conchas_532nm_LIT6_n10_1000ms_0.15mm_50p_cara']

Directorio 09042024-160856_conchas_532nm_LIT1_n10_1000ms_0.05mm_50p_atras contiene ['AfterPhoto.png', 'autoRaman_v24.m', 'BeforePhoto.png', 'experiment.txt', 'exported_preproc_exported.csv', 'exported_raw_exported.csv', 'matlabData.mat', 'polygon.mat', 'procesaRaman_v17_18042024-120353.m', 'recorded_path.mat', 'stitchedImage.fig', 'stitchedImage.png', 'ventana_ratio.png']
Directorio 10042024-103809_conchas_532nm_LIT2_n10_1000ms_0.05mm_50p_atras contiene ['AfterPhoto.png', 'autoRaman_v25.m', 'BeforePhoto.png', 'experiment.txt', 'exported_preproc_exported.csv', 'exported_raw_exported.csv', 'matlabData.mat', 'polygon.mat', 'procesaRaman_v17_10042024-135832.m', 'recorded_path.mat', 'stitchedImage.fig', 'stitchedImage.png', 'ventana_ratio.png']
Directorio 10042024-152046_conchas_532nm_LIT3_n10_1000ms_0.05mm_50_atras contiene ['AfterPhoto.png', 'autoRaman_v25.m', 'BeforePhoto.png', 'experiment.txt', 'exported_preproc_exported.csv', 'exported_raw_exported.csv', 'matlabData.mat', 'polygon.mat'

In [ ]:
def create_df_raw(folder_path, folder_names):
    dataframes = []
    lista_archivos = []
    lista_muestras = []
    background = []
    for folder in folder_names:
        path = os.path.join(folder_path, folder)
        if not os.path.exists(os.path.join(path, 'exported_raw_exported.csv')):
            print('No exported files found in', folder)
            continue
        print('Processing', folder)
        csv_files = [f for f in os.listdir(path) if f.endswith('exported_raw_exported.csv')]

        # Lee los archivos csv y los concatena en un solo DataFrame
        for csv_file in csv_files:
            df = pd.read_csv(os.path.join(path, csv_file), skiprows=[1], header=0)
            bg = pd.read_csv(os.path.join(path, csv_file), nrows=1)
            # print((os.path.join(path, csv_file)))
            nombre_archivo = folder+'_'+ csv_file.split("_")[1] # Nombre del archivo
            nombre_muestra = folder.split("_")[2] # Nombre de la muestra
            fecha_medida = folder.split("_")[0].split("-")[0]   # Fecha de la medida
            hora_medida = folder.split("_")[0].split("-")[1]    # Hora de la medida
            tipo = csv_file.split("_")[1]#.split("-")[0]    # Tipo de medida (preproc, raw)
            df.index = pd.MultiIndex.from_tuples([( nombre_muestra, tipo, fecha_medida,hora_medida, nombre_archivo, X, Y, ) for X, Y in df.index],
                                                   names=[ "Muestra", "Tipo", "Fecha","Hora", "Archivo","X", "Y"])
            # Guardamos el nombre de los archivos y de las muestras en lsitas
            lista_archivos.append(nombre_archivo)
            lista_muestras.append(nombre_muestra)
            # Agrega el DataFrame a la lista

            # Preprocesamos el DataFrame 
            preprocesado_raw(df,bg)
            dataframes.append(df)
            background.append(bg)

    # Concatena todos los DataFrames en uno solo
    data = pd.concat(dataframes)
    return lista_archivos, lista_muestras

def preprocesado_raw(df,bg):
    # Restamos el background
    df = df.sub(bg.iloc[0], axis=1)
    # Normalizamos
    df = df.div(df.max(axis=1), axis=0)
    return df

In [19]:
tipo_medida = 'all'
for folder in directories:
                path = os.path.join(folder_path, folder)
                if not os.path.exists(os.path.join(path, 'exported_preproc_exported.csv')) and not os.path.exists(os.path.join(path, 'exported_raw_exported.csv')):
                    print('No exported files found in', folder)
                    continue
                print('Processing', folder)
                # Lista de archivos en el directorio en función del tipo (all, exported_preproc_exported, exported_raw_exported)
                if tipo_medida == "all":
                    csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
                    print(csv_files)
                elif tipo_medida == "exported_preproc_exported" or tipo_medida == "exported_raw_exported":
                    csv_files = [f for f in os.listdir(path) if f.endswith(f'{tipo_medida}.csv')]
                    print(csv_files)

                for csv_file in csv_files:
                    print((os.path.join(path, csv_file)))

No exported files found in 09042024-160856_conchas_532nm_LIT1_n10_1000ms_0.05mm_50p_atras
Processing 10042024-103809_conchas_532nm_LIT2_n10_1000ms_0.05mm_50p_atras
['exported_preproc_exported.csv', 'exported_raw_exported.csv']
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\10042024-103809_conchas_532nm_LIT2_n10_1000ms_0.05mm_50p_atras\exported_preproc_exported.csv
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\10042024-103809_conchas_532nm_LIT2_n10_1000ms_0.05mm_50p_atras\exported_raw_exported.csv
Processing 10042024-152046_conchas_532nm_LIT3_n10_1000ms_0.05mm_50_atras
['exported_preproc_exported.csv', 'exported_raw_exported.csv']
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\10042024-152046_conchas_532nm_LIT3_n10_1000ms_0.05mm_50_atras\exported_preproc_exported.csv
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\10042024-152046_conchas_532nm_LIT3_n10_1000ms_0.05mm_50_atras\exported_raw_exported.csv
Processing 11042024-162047_conchas_532nm_LIT4_n10_1000ms_0.05mm_50p_caraabajo
['exported_preproc_export

In [28]:
def create_df_sonda_raw(folder_names, folder_path = "Data"):
    dataframes = []
    lista_archivos = []
    lista_muestras = []
    background = []
    for folder in folder_names:
        path = os.path.join(folder_path, folder)

        # Lista de archivos en el directorio en función del tipo (all, exported_preproc_exported, exported_raw_exported)
        if type == "all":
            csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
        elif type == "exported_preproc_exported" or type == "exported_raw_exported":
            csv_files = [f for f in os.listdir(path) if f.endswith(f'{type}.csv')]

        # Lee los archivos csv y los concatena en un solo DataFrame
        for csv_file in csv_files:
            # df = pd.read_csv(os.path.join(path, csv_file))
            print((os.path.join(path, csv_file)))
            # nombre_archivo = folder+'_'+ csv_file.split("_")[1] # Nombre del archivo
            # nombre_muestra = folder.split("_")[2] # Nombre de la muestra
            # fecha_medida = folder.split("_")[0].split("-")[0]   # Fecha de la medida
            # hora_medida = folder.split("_")[0].split("-")[1]    # Hora de la medida
            # tipo = csv_file.split("_")[1]#.split("-")[0]    # Tipo de medida (preproc, raw)
            # # df.index = pd.MultiIndex.from_tuples([( nombre_muestra, tipo, fecha_medida,hora_medida, nombre_archivo, X, Y, ) for X, Y in df.index],
            # #                                       names=[ "Muestra", "Tipo", "Fecha","Hora", "Archivo","X", "Y"])
            # # Guardamos el nombre de los archivos y de las muestras en lsitas
            # lista_archivos.append(nombre_archivo)
            # lista_muestras.append(nombre_muestra)
            # # Agrega el DataFrame a la lista
            # # dataframes.append(df)

    # Concatena todos los DataFrames en uno solo
    # data = pd.concat(dataframes)
    return lista_archivos, lista_muestras

lista_archivos, lista_muestras = create_df_sonda(directories_test, folder_path, "exported_raw_exported")



D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\09042024-160856_conchas_532nm_LIT1_n10_1000ms_0.05mm_50p_atras\exported_raw_exported.csv
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\10042024-103809_conchas_532nm_LIT2_n10_1000ms_0.05mm_50p_atras\exported_raw_exported.csv
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\10042024-152046_conchas_532nm_LIT3_n10_1000ms_0.05mm_50_atras\exported_raw_exported.csv
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\11042024-162047_conchas_532nm_LIT4_n10_1000ms_0.05mm_50p_caraabajo\exported_raw_exported.csv
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\17042024-150606_conchas_532nm_LIT5_n10_1000ms_0.05mm_50p_atras\exported_raw_exported.csv
D:/OneDrive - UNICAN/LIBS/raman/ConchasArq\17042024-184049_conchas_532nm_LIT6_n10_1000ms_0.15mm_50p_cara\exported_raw_exported.csv


In [71]:

df_background = pd.read_csv('D:/OneDrive - UNICAN/LIBS/raman/ConchasArq/09042024-160856_conchas_532nm_LIT1_n10_1000ms_0.05mm_50p_atras/exported_raw_exported.csv', nrows=1)

df = pd.read_csv('D:/OneDrive - UNICAN/LIBS/raman/ConchasArq/09042024-160856_conchas_532nm_LIT1_n10_1000ms_0.05mm_50p_atras/exported_raw_exported.csv', skiprows=[1], header=0)


In [73]:
df_background

,-27.19,-23.36,-19.54,-15.72,-11.9,-8.08,-4.27,-0.46,3.35,7.16,...,2919.78,2921.86,2923.95,2926.03,2928.11,2930.19,2932.27,2934.34,2936.42,2938.5
0,1627.88,1621.13,1619.38,1618.25,1629,1632.25,1629.25,1634.5,1633.63,1636.13,...,1643,1645.63,1640.5,1637.88,1637.38,1635.13,1619.5,1619.38,1618.88,1618.25


,,-27.19,-23.36,-19.54,-15.72,-11.9,-8.08,-4.27,-0.46,3.35,7.16,...,2919.78,2921.86,2923.95,2926.03,2928.11,2930.19,2932.27,2934.34,2936.42,2938.5
-6.186,4.347,1628.1,1622.5,1618.4,1617.5,1668.9,1738.0,2028.6,2227.2,2014.7,1731.5,...,10626.1,10509.2,10370.4,10243.3,10105.0,9957.2,1620.1,1619.4,1618.5,1618.6
-6.136,4.347,1628.0,1620.0,1618.1,1617.2,1664.5,1724.8,2007.5,2228.4,2036.6,1738.0,...,10042.7,9933.5,9815.1,9702.2,9561.9,9433.1,1620.7,1619.2,1618.6,1617.4
-6.086,4.347,1627.4,1621.2,1619.1,1617.9,1667.4,1729.5,2024.1,2259.4,2067.1,1749.1,...,10517.4,10406.1,10287.1,10133.1,10012.3,9892.9,1620.7,1619.9,1619.0,1618.8
-6.036,4.347,1626.3,1621.5,1618.1,1617.3,1664.3,1721.1,2014.5,2274.2,2095.5,1755.6,...,10129.6,10005.2,9905.2,9737.7,9643.7,9499.0,1620.8,1619.1,1618.4,1618.3
-5.986,4.347,1627.0,1620.4,1618.5,1618.3,1662.4,1718.1,2009.1,2271.8,2086.5,1746.9,...,9440.4,9327.9,9187.7,9116.2,8992.1,8875.4,1620.1,1618.5,1618.7,1619.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-4.686,5.347,1627.7,1620.5,1618.4,1619.3,1659.8,1713.9,2014.2,2303.9,2121.7,1752.6,...,9011.2,8888.2,8790.7,8683.0,8576.2,8469.5,1619.5,1619.0,1618.0,1620.0
-4.636,5.347,1627.6,1620.5,1619.8,1617.9,1660.1,1719.5,2054.0,2388.9,2196.2,1774.5,...,8897.5,8803.3,8694.4,8598.3,8500.4,8361.5,1620.5,1619.4,1618.2,1619.3
-4.586,5.347,1628.4,1622.0,1618.2,1618.3,1661.7,1715.9,2007.7,2295.5,2101.1,1753.2,...,9237.2,9158.4,9039.9,8922.3,8809.1,8714.2,1620.4,1619.9,1618.3,1618.8
-4.536,5.347,1627.7,1621.5,1617.4,1617.3,1660.9,1712.6,1999.7,2297.4,2112.6,1755.4,...,9817.6,9710.9,9584.0,9446.1,9339.9,9216.4,1619.4,1619.6,1619.4,1619.1


In [65]:
df_background

,-27.19,-23.36,-19.54,-15.72,-11.9,-8.08,-4.27,-0.46,3.35,7.16,...,2919.78,2921.86,2923.95,2926.03,2928.11,2930.19,2932.27,2934.34,2936.42,2938.5
0,1627.88,1621.13,1619.38,1618.25,1629,1632.25,1629.25,1634.5,1633.63,1636.13,...,1643,1645.63,1640.5,1637.88,1637.38,1635.13,1619.5,1619.38,1618.88,1618.25


In [68]:
header

,0,1,2,3,4,5,6,7,8,9,...,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043
0,-27.19,-23.36,-19.54,-15.72,-11.9,-8.08,-4.27,-0.46,3.35,7.16,...,2919.78,2921.86,2923.95,2926.03,2928.11,2930.19,2932.27,2934.34,2936.42,2938.5


In [56]:
primera_fila = df_background.iloc[0]
primera_columna = primera_fila.index[0]


-27.19     1627.88
-23.36     1621.13
-19.54     1619.38
-15.72     1618.25
-11.9      1629.00
            ...   
2930.19    1635.13
2932.27    1619.50
2934.34    1619.38
2936.42    1618.88
2938.5     1618.25
Name: 0, Length: 1044, dtype: float64